In [1]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = (8.0, 8.0)
import CameraNetwork
import CameraNetwork.global_settings as gs
import cv2
import Image
import ImageDraw
import time
from tqdm import tqdm_notebook
import traceback

c:\users\user\documents\code\cameranetwork_git\CameraNetwork\transformation_matrices.py:1899: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


In [2]:
c = CameraNetwork.CLIclient()
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
SERVERS = c.client_instance.servers
print SERVERS

['109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119']


In [3]:
CAM = 113
DAY = 5
MONTH = 4

In [4]:
df = c.query('{}'.format(CAM), '{:02}/{}/2017'.format(MONTH, DAY))[0]
print("Camera {} captured {} images on this day.".format(CAM, df.shape[0]))

Camera 113 captured 668 images on this day.


In [5]:
JPEG_QUALITY = 100
video = cv2.VideoWriter(
    'video_{}_2017_{:02}_{:02}.avi'.format(CAM, MONTH, DAY),
    -1, 6, (301, 301)
)

for ind in tqdm_notebook(df.loc[(slice(None), '2'), :].index):
    try:
        img = c.seek('{}'.format(CAM), ind[0], ind[1], JPEG_QUALITY, 301)[0][0]
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        draw.text(
            (0, 0),
            ind[0].time().strftime("%H:%M:%S"),
            (255,255,255)
        )

        video.write(cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
    except Exception, e:
        traceback.print_exc()
        
video.release()

In [8]:
video.release()

In [16]:
c.seek('{}'.format(CAM), ind[0], ind[1], JPEG_QUALITY, 301)[0]

[array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ..., 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8)]